<a href="https://colab.research.google.com/github/mevangelista-alvarado/mx_municipally_network/blob/main/MX_municipality_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download

### Via manual

Download the files [here](http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463142683_s.zip)

You must upload the following files in the `/content/conjunto_de_datos` dir:
  *   `areas_geoestadisticas_municipales.CPG`
  *   `areas_geoestadisticas_municipales.dbf`
  *   `areas_geoestadisticas_municipales.prj`
  *   `areas_geoestadisticas_municipales.shp`
  *   `areas_geoestadisticas_municipales.shx`


### Via console

In [ ]:
!wget http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463142683_s.zip

In [ ]:
import zipfile

path_to_zip_file = '/content/889463142683_s.zip'
directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)


## Create a MX municipality dataframe

### Download geopandas module

In [ ]:
!pip install geopandas

### Find the neighbor for each municipality

In [ ]:
import geopandas as gpd
from itertools import zip_longest

# Read shp file and convert as dataframe
path_file = "/content/conjunto_de_datos/areas_geoestadisticas_municipales.shp"
df = gpd.read_file(path_file)

# Create CVE_ENT+MUN column to avoid confusion to make the Mexico network
CVE_Ent = df["CVE_ENT"]
CVE_Mun = df["CVE_MUN"]
CVE_Ent_Mun = ["".join(CVE) for CVE in zip_longest(CVE_Ent, CVE_Mun, fillvalue=0)]
df["CVE_Ent_Mun"] = CVE_Ent_Mun


# Create representative_point list
# ref: https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.representative_point.html
df["representative_point"] = [e for e in df.representative_point()]

# Create the adjacent column to save all nodes that are neighbors from a specific node 
df = df.assign(adjacents=None)

# Find the neighbor for each municipality
for index, municipality in df.iterrows():
  # Create a list with all neighbor from a municipality (CVE_Ent_Mun := ID of the State + Municipality )
  adjacents = df[-df.geometry.disjoint(municipality.geometry)].CVE_Ent_Mun.tolist()
  # Remove own name from the list
  adjacents = [name for name in adjacents if municipality.CVE_Ent_Mun != name]
  # Add the result in the adjacents column
  df.at[index, "adjacents"] = ", ".join(adjacents) 

# Show dataframe head 
df.head()


## Create a MX municipality network 

#### Define dictionaries to add metadata to each node

In [ ]:
# Generates dictionaries to add metadata to each node (aka CVE_Ent_Mun)
CVE = df["CVE_Ent_Mun"]
representative_point = df["representative_point"]
MUN_NAME = df["NOM_MUN"]

# Dictionaries
# Example {'01002': (2490227.155643021 1123688.502500003)}
# Where the keys are str type and the values are point type
fixed_positions = dict(zip(CVE, representative_point))  
# Example {'01002': 'Asientos'}
# Where the keys and values are str type
mun_name = dict(zip(CVE, MUN_NAME)) 

In [ ]:
# This dictionary is to use assign the state name to in each node 
STATE_NAME = {
  '01':	'AGUASCALIENTES',
  '02':	'BAJA CALIFORNIA',
  '03':	'BAJA CALIFORNIA SUR',
  '04':	'CAMPECHE',
  '05':	'COAHUILA DE ZARAGOZA',
  '06':	'COLIMA',
  '07':	'CHIAPAS',
  '08':	'CHIHUAHUA',
  '09':	'CIUDAD DE MÉXICO',
  '10':	'DURANGO',
  '11':	'GUANAJUATO',
  '12':	'GUERRERO',
  '13':	'HIDALGO',
  '14':	'JALISCO',
  '15':	'EDO DE MÉXICO',
  '16':	'MICHOACÁN DE OCAMPO',
  '17':	'MORELOS',
  '18':	'NAYARIT',
  '19':	'NUEVO LEÓN',
  '20':	'OAXACA',
  '21':	'PUEBLA',
  '22':	'QUERÉTARO',
  '23':	'QUINTANA ROO',
  '24':	'SAN LUIS POTOSÍ',
  '25':	'SINALOA',
  '26':	'SONORA',
  '27':	'TABASCO',
  '28':	'TAMAULIPAS',
  '29':	'TLAXCALA',
  '30':	'VERACRUZ DE IGNACIO DE LA LLAVE',
  '31':	'YUCATÁN',
  '32':	'ZACATECAS',
}

### Create network and assign metadata to each node

In [ ]:
import networkx as nx

# Define the list to use to create the MX network
mun = df['CVE_Ent_Mun'].tolist()
neighbors = df['adjacents'].tolist()

# Create a empty newtwork
G = nx.Graph()

for mun_id in mun:
  # Add each municipality with metadata as a node in the network 
  G.add_node(mun_id,
              pos=fixed_positions[mun_id],
              name=mun_name[mun_id],
              state=STATE_NAME[mun_id[0:2]],
  )
  # Add all edge from the node
  index = mun.index(mun_id)
  neighbors_by_mun = neighbors[index]
  _neighbors = [i.strip() for i in neighbors_by_mun.split(',')]
  for neighbor in _neighbors:
    G.add_edge(mun_id, neighbor)
  # Add Municipality name

# Shows all nodes (qty, nodes)
# print("Nodos: ", G.number_of_nodes(), G.nodes())
# Shows all edges (qty, edges)
# print("Enlaces: ", G.number_of_edges(),G.edges())

In [ ]:
# Show the node data
G.nodes(data=True)

## MX municipality network visualization 

We use the plotly module

In [ ]:
import plotly.graph_objects as go

### Add all edge from the network to show in the chart 

In [ ]:
edge_x = list()
edge_y = list()
# Iterate over edges
for edge in G.edges():
  pt0 = G.nodes[edge[0]]["pos"]
  pt1 = G.nodes[edge[1]]["pos"]
  edge_x += [pt0.x, pt1.x, None]
  edge_y += [pt0.y, pt1.y, None]

# Create a Scatter object with the edges from the Mx municipality network 
edge_trace = go.Scatter(
  x=edge_x, y=edge_y,
  line=dict(width=0.5, color='blue'),
  hoverinfo='none',
  mode='lines',
)

###  Add all nodes from the network to show in the chart

In [ ]:
node_x = list()
node_y = list()
_text = list()
# Iterate over edges
for node in G.nodes():
  # Get the node point, municipality name, and state name 
  pt = G.nodes[node]["pos"]
  mun_name = G.nodes[node]["name"]
  state_name = G.nodes[node]["state"]
  # Save the data in the previous lists to use in the Scatter object 
  node_x.append(pt.x)
  node_y.append(pt.y)
  _text.append(f'Municipio: {mun_name},\nEstado: {state_name}')
  # Remark: In this for is it can possible the added color to each node

# Create a Scatter object with the edges from the Mx municipality network 
node_trace = go.Scatter(
  x=node_x, y=node_y,
  text=_text,
  mode='markers',
  hoverinfo='text',
  marker=dict(
      color=['black'],
      line_width=1,
  )
)

### Create the MX municipality network plot  

In [ ]:
# Ploty figure
fig = go.Figure(
  data=[edge_trace, node_trace],
  layout=go.Layout(
    title="MX municipality network",
    titlefont_size=18,
    template='plotly_white',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    annotations=[dict(
        text="Python code: <a href='https://github.com/mevangelista-alvarado/mx_municipally_network'> https://github.com/mevangelista-alvarado/mx_municipally_network</a>",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=0.005,
        y=-0.002,
    )],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
  )
)
fig.show()